# Project 13 -- Anthony Micco

**TA Help:** N/A

**Collaboration:** N/A

**Internet Resources:** Used Stack Overflow to help me determine I needed to use HAVING instead of WHERE when working with already aggregated data (https://stackoverflow.com/questions/42470849/why-are-aggregate-functions-not-allowed-in-where-clause)

**ChatGPT, Gemini, Claude, etc:** None

**Link to AI Chat History**: None

**OVERALL MESSAGE:** Any time that you used anything except your brain to solve the questions in these projects, you need to disclose such resources at the start of the project, with details about your usage of the tools.

**YOUR OWN WORK:** Even when you utilize other resources, do NOT just copy and paste.  Write all explanations in your own words, using several sentences in English, which are understandable and which you wrote (and did not just copy and paste).

## Question 1

In [17]:
#code to initialize a SparkSession
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("TDM_S").config("spark.driver.memory", "2g").getOrCreate()
# Initialize a SparkSession called 'TDM_S' with 2GB of memory

In [18]:
#loading in the weather.parquet dataset 
df = spark.read.parquet("/anvil/projects/tdm/data/whin/weather.parquet")

In [19]:
#showing the first 5 lines of the dataset
df.show(5)

+----------+--------+---------+---------------+--------------------+-----------+----------------+---------------+--------+---------------+--------------------+----+---------------------+--------------+----------------------+-------------------+---------------------------+--------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+
|station_id|latitude|longitude|           name|    observation_time|temperature|temperature_high|temperature_low|humidity|solar_radiation|solar_radiation_high|rain|rain_inches_last_hour|wind_speed_mph|wind_direction_degrees|wind_gust_speed_mph|wind_gust_direction_degrees|pressure|soil_temp_1|soil_temp_2|soil_temp_3|soil_temp_4|soil_moist_1|soil_moist_2|soil_moist_3|soil_moist_4|
+----------+--------+---------+---------------+--------------------+-----------+----------------+---------------+--------+---------------+--------------------+----+---------------------+--------------+----------------------+------------

## Question 2

In [20]:
# printing out basic info about the dataframe
df.printSchema()

root
 |-- station_id: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- observation_time: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- temperature_high: double (nullable = true)
 |-- temperature_low: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- solar_radiation: double (nullable = true)
 |-- solar_radiation_high: double (nullable = true)
 |-- rain: double (nullable = true)
 |-- rain_inches_last_hour: double (nullable = true)
 |-- wind_speed_mph: double (nullable = true)
 |-- wind_direction_degrees: double (nullable = true)
 |-- wind_gust_speed_mph: double (nullable = true)
 |-- wind_gust_direction_degrees: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- soil_temp_1: double (nullable = true)
 |-- soil_temp_2: double (nullable = true)
 |-- soil_temp_3: double (nullable = true)
 |-- soil_temp_4: double (nullable = true)
 |

In [25]:
#using the example to test the function of PySpark
test_df = df.groupBy("station_id").agg(avg("wind_speed_mph").alias("average_wind_speed_mph"), max("temperature").alias("max_temperature")).filter(col("max_temperature") > 80).sort(desc("average_wind_speed_mph"))
test_df.show(5)

+----------+----------------------+---------------+
|station_id|average_wind_speed_mph|max_temperature|
+----------+----------------------+---------------+
|       167|     8.294951993017166|           93.0|
|        85|     7.560077632217371|           94.0|
|       145|     7.392609344809415|           95.0|
|       109|      7.25007006071929|           93.0|
|        81|     7.246332637648083|          158.0|
+----------+----------------------+---------------+
only showing top 5 rows


In [23]:
# operations to aggregate, filter, sort and group the data
import time
start_time = time.time()
result_df = df.groupBy("name").agg(avg("humidity").alias("avg_humidity"),avg("solar_radiation").alias("avg_solar_radiation"),avg("pressure").alias("avg_pressure")).filter((col("avg_humidity") > 50) &  (col("avg_solar_radiation") > 100)).sort(desc("avg_pressure"))
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

Time taken: 0.026845216751098633 seconds


In [24]:
#printing out the first five rows of that data set
start_time = time.time()
result_df.show(5)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

+----------------+-----------------+-------------------+------------------+
|            name|     avg_humidity|avg_solar_radiation|      avg_pressure|
+----------------+-----------------+-------------------+------------------+
|WHIN089E-BENT009|74.77029578351164| 257.34197608558844|30.216278791692908|
|      Bringhurst|80.17502311929556| 168.08889779559118|30.190432274525733|
| WHIN020-FOUN001|76.72776597425857| 146.12622262805766|30.159863727349002|
|        Idaville|77.37444567283207| 166.06921923967843|30.151303970200168|
|         Wolcott|67.00495091507085| 104.93433155080214| 30.11409279198035|
+----------------+-----------------+-------------------+------------------+
only showing top 5 rows
Time taken: 1.964643955230713 seconds


## Question 3

In [37]:
# creating a temporary view of the dataframe
df.createOrReplaceTempView("weather_view")

In [38]:
#writing an SQL query using PySpark function
test_df = spark.sql("SELECT AVG(humidity), AVG(solar_radiation), AVG(pressure) FROM weather_view")

In [39]:
#showing the first five rows
test_df.show(5)

+----------------+--------------------+------------------+
|   avg(humidity)|avg(solar_radiation)|     avg(pressure)|
+----------------+--------------------+------------------+
|77.5701237585927|  183.93572113847137|29.912132181425918|
+----------------+--------------------+------------------+



In [40]:
#using SQL with PySpark to find the same operations as the previous question
start_time = time.time()
result_df = spark.sql("""
SELECT name, 
AVG(humidity) AS avg_humidity,
AVG(solar_radiation) AS avg_solar_radiation,
AVG(pressure) AS avg_pressure
FROM weather_view
GROUP BY name
HAVING (avg_humidity > 50) AND (avg_solar_radiation > 100)
ORDER BY avg_pressure DESC
""")
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

Time taken: 0.012638092041015625 seconds


In [41]:
#determing how much time it takes to run
start_time = time.time()
result_df.show(5)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

+----------------+-----------------+-------------------+------------------+
|            name|     avg_humidity|avg_solar_radiation|      avg_pressure|
+----------------+-----------------+-------------------+------------------+
|WHIN089E-BENT009|74.77029578351164| 257.34197608558844|30.216278791692908|
|      Bringhurst|80.17502311929556| 168.08889779559118|30.190432274525733|
| WHIN020-FOUN001|76.72776597425857| 146.12622262805766|30.159863727349002|
|        Idaville|77.37444567283207| 166.06921923967843|30.151303970200168|
|         Wolcott|67.00495091507085| 104.93433155080214| 30.11409279198035|
+----------------+-----------------+-------------------+------------------+
only showing top 5 rows
Time taken: 0.953681230545044 seconds


3.4) It takes 0.953681230545044 seconds to print the first 5 rows of the result DataFrame \
3.5) It is a little over a second faster to print the first 5 rows of the result using PySpark SQL when compared to using the traditional PySpark DataFrame API. It took PySpark SQL only 0.953681230545044 seconds, while the PySpark DataFrame API took 1.964643955230713 seconds. 

## Question 4

In [43]:
#how to create a new column where every value in the new column is one more than the value in the old column
newdf = df.withColumn("new_column", col("old_column") + 1)

{"ts": "2025-11-18 15:35:30.010", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `old_column` cannot be resolved. Did you mean one of the following? [`latitude`, `longitude`, `humidity`, `name`, `pressure`]. SQLSTATE: 42703", "context": {"file": "line 2 in cell [43]", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o283.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `old_column` cannot be resolved. Did you mean one of the following? [`latitude`, `longitude`, `humidity`, `name`, `pressure`]. SQLSTATE: 42703;\n'Project [station_id#392L, latitude#393, longitude#394, name#395, observation_time#396, temperature#397, temperature_high#398, temperature_low#399, humidity#400, solar

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `old_column` cannot be resolved. Did you mean one of the following? [`latitude`, `longitude`, `humidity`, `name`, `pressure`]. SQLSTATE: 42703;
'Project [station_id#392L, latitude#393, longitude#394, name#395, observation_time#396, temperature#397, temperature_high#398, temperature_low#399, humidity#400, solar_radiation#401, solar_radiation_high#402, rain#403, rain_inches_last_hour#404, wind_speed_mph#405, wind_direction_degrees#406, wind_gust_speed_mph#407, wind_gust_direction_degrees#408, pressure#409, soil_temp_1#410, soil_temp_2#411, soil_temp_3#412, soil_temp_4#413, soil_moist_1#414, soil_moist_2#415, soil_moist_3#416, ... 2 more fields]
+- Relation [station_id#392L,latitude#393,longitude#394,name#395,observation_time#396,temperature#397,temperature_high#398,temperature_low#399,humidity#400,solar_radiation#401,solar_radiation_high#402,rain#403,rain_inches_last_hour#404,wind_s

In [44]:
#how to remove a column from a dataframe
newdf = df.drop("column_name")

In [46]:
#how to change the data type of a column by either creating a new column or overwriting an existing one
newdf = df.withColumn("new_column", col("old_column").cast("string"))

newdf = df.withColumn("old_column", col("old_column").cast("string"))

{"ts": "2025-11-18 15:37:25.698", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `old_column` cannot be resolved. Did you mean one of the following? [`latitude`, `longitude`, `humidity`, `name`, `pressure`]. SQLSTATE: 42703", "context": {"file": "line 2 in cell [46]", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o283.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `old_column` cannot be resolved. Did you mean one of the following? [`latitude`, `longitude`, `humidity`, `name`, `pressure`]. SQLSTATE: 42703;\n'Project [station_id#392L, latitude#393, longitude#394, name#395, observation_time#396, temperature#397, temperature_high#398, temperature_low#399, humidity#400, solar

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `old_column` cannot be resolved. Did you mean one of the following? [`latitude`, `longitude`, `humidity`, `name`, `pressure`]. SQLSTATE: 42703;
'Project [station_id#392L, latitude#393, longitude#394, name#395, observation_time#396, temperature#397, temperature_high#398, temperature_low#399, humidity#400, solar_radiation#401, solar_radiation_high#402, rain#403, rain_inches_last_hour#404, wind_speed_mph#405, wind_direction_degrees#406, wind_gust_speed_mph#407, wind_gust_direction_degrees#408, pressure#409, soil_temp_1#410, soil_temp_2#411, soil_temp_3#412, soil_temp_4#413, soil_moist_1#414, soil_moist_2#415, soil_moist_3#416, ... 2 more fields]
+- Relation [station_id#392L,latitude#393,longitude#394,name#395,observation_time#396,temperature#397,temperature_high#398,temperature_low#399,humidity#400,solar_radiation#401,solar_radiation_high#402,rain#403,rain_inches_last_hour#404,wind_s

In [47]:
#converting the observation_time column to a timestamp type
newdf = df.withColumn("observation_time", to_timestamp("observation_time"))

In [48]:
#storing the year of the timestamp in a new column
newdf = newdf.withColumn("year", year("observation_time"))

In [49]:
#storing the month of the timestamp in a new column
newdf = newdf.withColumn("month", month("observation_time"))

In [51]:
#displaying the new columns
newdf.printSchema()

root
 |-- station_id: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- observation_time: timestamp (nullable = true)
 |-- temperature: double (nullable = true)
 |-- temperature_high: double (nullable = true)
 |-- temperature_low: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- solar_radiation: double (nullable = true)
 |-- solar_radiation_high: double (nullable = true)
 |-- rain: double (nullable = true)
 |-- rain_inches_last_hour: double (nullable = true)
 |-- wind_speed_mph: double (nullable = true)
 |-- wind_direction_degrees: double (nullable = true)
 |-- wind_gust_speed_mph: double (nullable = true)
 |-- wind_gust_direction_degrees: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- soil_temp_1: double (nullable = true)
 |-- soil_temp_2: double (nullable = true)
 |-- soil_temp_3: double (nullable = true)
 |-- soil_temp_4: double (nullable = true)

In [52]:
# grouping the data by the month and then aggregating the temperature and solar_radiation by the average

#creating a new view for the new dataframe we created
newdf.createOrReplaceTempView("date_view")

result_df = spark.sql("""
    SELECT 
    month,
    AVG(temperature) AS avg_temp,
    AVG(solar_radiation) AS avg_solar_radiation
    FROM date_view
    GROUP BY month
    ORDER BY avg_temp DESC
""")

In [53]:
#printing the top 5 results
result_df.show(5)

+-----+-----------------+-------------------+
|month|         avg_temp|avg_solar_radiation|
+-----+-----------------+-------------------+
|    7| 73.9766158271431| 253.92342957199992|
|    6|73.41862618893836| 267.88055625964927|
|    8|71.00368368101219| 253.58956703962878|
|    9|66.70186356073211|  179.6384488924556|
|    5|60.18404719420269|  232.1310546777555|
+-----+-----------------+-------------------+
only showing top 5 rows


## Question 5

In [55]:
# using the explain method on code from question 4
result_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [avg_temp#854 DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(avg_temp#854 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=420]
      +- HashAggregate(keys=[month#853], functions=[avg(temperature#397), avg(solar_radiation#401)])
         +- Exchange hashpartitioning(month#853, 200), ENSURE_REQUIREMENTS, [plan_id=417]
            +- HashAggregate(keys=[month#853], functions=[partial_avg(temperature#397), partial_avg(solar_radiation#401)])
               +- Project [temperature#397, solar_radiation#401, month(cast(cast(observation_time#396 as timestamp) as date)) AS month#853]
                  +- FileScan parquet [observation_time#396,temperature#397,solar_radiation#401] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/anvil/projects/tdm/data/whin/weather.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<observation_time:string,temperature:double

In [58]:
#using mode=cost to print out the Optimized Logical Plan
result_df.explain(mode="cost")

== Optimized Logical Plan ==
Sort [avg_temp#854 DESC NULLS LAST], true, Statistics(sizeInBytes=9.9 MiB)
+- Aggregate [month#853], [month#853, avg(temperature#397) AS avg_temp#854, avg(solar_radiation#401) AS avg_solar_radiation#855], Statistics(sizeInBytes=9.9 MiB)
   +- Project [temperature#397, solar_radiation#401, month(cast(cast(observation_time#396 as timestamp) as date)) AS month#853], Statistics(sizeInBytes=9.9 MiB)
      +- Relation [station_id#392L,latitude#393,longitude#394,name#395,observation_time#396,temperature#397,temperature_high#398,temperature_low#399,humidity#400,solar_radiation#401,solar_radiation_high#402,rain#403,rain_inches_last_hour#404,wind_speed_mph#405,wind_direction_degrees#406,wind_gust_speed_mph#407,wind_gust_direction_degrees#408,pressure#409,soil_temp_1#410,soil_temp_2#411,soil_temp_3#412,soil_temp_4#413,soil_moist_1#414,soil_moist_2#415,soil_moist_3#416,... 1 more fields] parquet, Statistics(sizeInBytes=85.1 MiB)

== Physical Plan ==
AdaptiveSparkPlan i

In [63]:
#creating a long, complex SQL query that has many steps
result_df = spark.sql("""
SELECT station_id, 
AVG(humidity) AS avg_humidity,
AVG(solar_radiation) AS avg_solar_radiation,
AVG(pressure) AS avg_pressure,
MAX(rain) AS max_rain,
AVG(soil_temp_1+soil_temp_2+soil_temp_3+soil_temp_4) AS avg_soil_temp,
MIN(soil_moist_1+soil_moist_2+soil_moist_3+soil_moist_4) AS min_soil_moist
FROM weather_view
GROUP BY station_id
HAVING (avg_humidity > 50) AND (avg_solar_radiation > 100)
ORDER BY avg_pressure DESC
""")

In [65]:
result_df.explain(mode="cost")

== Optimized Logical Plan ==
Sort [avg_pressure#927 DESC NULLS LAST], true, Statistics(sizeInBytes=22.7 MiB)
+- Filter ((isnotnull(avg_humidity#925) AND isnotnull(avg_solar_radiation#926)) AND ((avg_humidity#925 > 50.0) AND (avg_solar_radiation#926 > 100.0))), Statistics(sizeInBytes=22.7 MiB)
   +- Aggregate [station_id#392L], [station_id#392L, avg(humidity#400) AS avg_humidity#925, avg(solar_radiation#401) AS avg_solar_radiation#926, avg(pressure#409) AS avg_pressure#927, max(rain#403) AS max_rain#928, avg((((soil_temp_1#410 + soil_temp_2#411) + soil_temp_3#412) + soil_temp_4#413)) AS avg_soil_temp#929, min((((soil_moist_1#414 + soil_moist_2#415) + soil_moist_3#416) + soil_moist_4#417)) AS min_soil_moist#930], Statistics(sizeInBytes=22.7 MiB)
      +- Project [station_id#392L, humidity#400, solar_radiation#401, rain#403, pressure#409, soil_temp_1#410, soil_temp_2#411, soil_temp_3#412, soil_temp_4#413, soil_moist_1#414, soil_moist_2#415, soil_moist_3#416, soil_moist_4#417], Statistics(

5.2) My physical plan had 8 steps, while my optimized logical plan had 4 steps.\
5.5) My physical plan had 8 steps, whle my optimized logical plan had 6 steps. \
5.6) Yes, it was much easier to understand the optimized logical plan because it used more identifiable language used in the SQL or API query such as sort and aggregate. It also made it easier to see which columns it was using for the query and did not include as much extra information. 

## Pledge

By submitting this work I hereby pledge that this is my own, personal work. I've acknowledged in the designated place at the top of this file all sources that I used to complete said work, including but not limited to: online resources, books, and electronic communications. I've noted all collaboration with fellow students and/or TA's. I did not copy or plagiarize another's work.

> As a Boilermaker pursuing academic excellence, I pledge to be honest and true in all that I do. Accountable together – We are Purdue.

https://www.purdue.edu/odos/osrr/honor-pledge/
